In [1]:
import time
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Map resources

In [2]:
class Food:
    def __init__(self, position):
        self.position = position
        self.type = 'item'


class Water:
    def __init__(self, position):
        self.position = position
        self.type = 'item'


class Garlic:
    def __init__(self, position):
        self.position = position
        self.type = 'item'

## Human

In [3]:
class Human:
    def __init__(self, name, health, age, position):
        self.name=name
        self.health = health
        self.age = age
        self.position = position
        self.is_turned = False
        self.type = 'human'

    def move(self, map_width, map_height):
        dx = random.randint(-4, 4)
        dy = random.randint(-4, 4)
        new_x = self.position[0] + dx
        new_y = self.position[1] + dy
        # Check if the new position is within the map bounds
        if 0 <= new_x < map_width and 0 <= new_y < map_height:
            self.position = (new_x, new_y)

    def interact_with_human(self, other):
        if random.random() < 0.4:
            # Selfish interaction: one human gains 20 health, the other loses 20 health
            self.health += 20
            other.health -= 20
            print(f"Selfish interaction: {self.name} gained 20 health, {other.name} lost 20 health")
        else:
            # Helpful interaction: both humans gain 10 health
            self.health += 10
            other.health += 10
            print(f"Helpful interaction: {self.name} and {other.name} gained 10 health each")

    def interact_with_vampire(self, vampire):
        if random.random() < 0.7:
            # Human is bitten by the vampire and becomes a vampire
            vampire.bite(self)
        else:
            # Human kills the vampire
            vampire.health = 0
            print(f"Human killed the vampire: {self.name} killed {vampire.name}")
            
    def interact_with_item(self, item):
        if isinstance(item, Food):
            self.health += 30
            print(f"{self.name} gained 30 health from food")
        elif isinstance(item, Water):
            self.health += 50
            print(f"{self.name} gained 50 health from water")
        elif isinstance(item, Garlic):
            self.health += 100
            print(f"{self.name} gained 100 health from garlic")

## Vampire

In [4]:
class Vampire:
    def __init__(self, name, initial_health, age, position):
        self.name = name
        self.health = initial_health
        self.age = random.randint(10, 50) if age == None else age
        self.position = position
        self.type = 'vampire'

    def move(self, map_width, map_height):
        dx = random.randint(-8, 8)
        dy = random.randint(-8, 8)
        new_x = self.position[0] + dx
        new_y = self.position[1] + dy
        # Check if the new position is within the map bounds
        if 0 <= new_x < map_width and 0 <= new_y < map_height:
            self.position = (new_x, new_y)

    def bite(self, human):
        # Human becomes a vampire
        vampire = Vampire(human.name, human.health, human.age, human.position)
        vampire.is_newly_turned = True
        print(f"Human bitten by the vampire: {human.name} turned into a vampire")

        # Remove the human from the simulation
        human.is_turned = True
        return vampire

    def interact_with_human(self, human):
        # Vampires only bite humans, no other interactions
        self.bite(human)

    def interact_with_vampire(self, vampire):
        # Vampires bite each other, both lose 20 health
        self.health -= 20
        vampire.health -= 20
        print(f"Vampire interaction: {self.name} and {vampire.name} both lost 20 health")
        
    def interact_with_item(self, item):
        if isinstance(item, Garlic):
            self.health -= 10
            print(f"{self.name} lost 10 health from garlic")


## Simulation 

In [5]:
class Simulation:
    def __init__(self, num_humans, num_vampires, num_timesteps, map_size=(10,10)):
        self.num_humans = num_humans
        self.num_vampires = num_vampires
        self.num_timesteps = num_timesteps
        self.map_size = map_size
        self.humans = []
        self.vampires = []
        self.food = []
        self.water = []
        self.garlic = []

        self.initialize_entities()

    def initialize_entities(self):
        for _ in range(self.num_humans):
            self.humans.append(Human(f"H{_}", 100, random.randint(10, 50), self.generate_random_position()))

        for _ in range(self.num_vampires):
            self.vampires.append(Vampire(f"V{_}", random.randint(80, 120), None, self.generate_random_position()))

        self.food = [Food(self.generate_random_position()) for _ in range(15)]
        self.water = [Water(self.generate_random_position()) for _ in range(8)]
        self.garlic = [Garlic(self.generate_random_position()) for _ in range(10)]

    def generate_random_position(self):
        x = random.randint(0, self.map_size[0])
        y = random.randint(0, self.map_size[1])
        return x, y

    def plot_map(self, title):
        fig, ax = plt.subplots(figsize=(10,10))

        # Plot humans
        if len(self.humans)>0:
            human_positions = [human.position for human in self.humans]
            ax.scatter(*zip(*human_positions), color='green', label='Humans')

        # Plot vampires
        if len(self.vampires)>0:
            vampire_positions = [vampire.position for vampire in self.vampires]
            ax.scatter(*zip(*vampire_positions), color='red', label='Vampires')

        # Plot food
        food_positions = [food.position for food in self.food]
        ax.scatter(*zip(*food_positions), color='brown', marker='s', label='Food')

        # Plot water
        water_positions = [water.position for water in self.water]
        ax.scatter(*zip(*water_positions), color='blue', s=100, marker='s', label='Water')

        # Plot garlic
        garlic_positions = [garlic.position for garlic in self.garlic]
        ax.scatter(*zip(*garlic_positions), color='gray', marker='^', label='Garlic')

        ax.legend(loc='best')
        #ax.legend(bbox_to_anchor=(0.4, 1.0), loc='lower left')
        
        #adjusting padding around plot
        #plt.xlim(-3, self.map_size[0]+3)
        #plt.ylim(-3, self.map_size[1]+3)
        #plt.tight_layout()
        #plt.show()

        #Add Title
        plt.title(title)
        
        # Convert the plot to a numpy array
        fig.canvas.draw()
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.close('all')
        return img

    def run_simulation(self):
        # Define the codec using VideoWriter_fourcc and create a VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        video = cv2.VideoWriter('simulation.mp4', fourcc, 1.0, (1000,1000))
        
        for timestep in range(self.num_timesteps):
            print(f"--- Timestep {timestep + 1} ---")
            self.move_entities()
            self.perform_interactions()
            frame = self.plot_map(f"{len(self.humans)}: Humans and {len(self.vampires)}: Vampires Left")
            video.write(frame)

            # Display the resulting frame 
            cv2.imshow('Frame', frame)
            time.sleep(0.3)
              
            # Press Q on keyboard to exit 
            if cv2.waitKey(25) & 0xFF == ord('q'): 
                break
                
            # Remove dead vampires
            self.vampires = [vampire for vampire in self.vampires if vampire.health > 0]

            # Add new vampires from converted humans
            new_vampires = [Vampire(f"V{human.name[1:]}_n" ,human.health, human.age, human.position) for human in self.humans if human.is_turned]
            self.vampires.extend(new_vampires)

            # Remove dead humans
            self.humans = [human for human in self.humans if human.health > 0]
            
            # Remove turned humans
            self.humans = [human for human in self.humans if not human.is_turned]
            
            print(f"{len(self.humans)}: Humans and {len(self.vampires)}: Vampires Left")
            if len(self.humans)==0 or len(self.vampires)==0:
                continue
                
        # Release the VideoWriter
        video.release()
          
        # Closes all the frames 
        cv2.destroyAllWindows() 

    def move_entities(self):
        for human in self.humans:
            human.move(self.map_size[0],self.map_size[1])

        for vampire in self.vampires:
            vampire.move(self.map_size[0],self.map_size[1])

    def perform_interactions(self):
        for human in self.humans:
            self.interact_with_others(human)

        for vampire in self.vampires:
            self.interact_with_others(vampire)

    def interact_with_others(self, entity):
        for other_entity in self.humans + self.vampires + self.food + self.water + self.garlic:
            if self.is_adjacent(entity.position, other_entity.position):
                if entity.type == 'human' and other_entity.type=='human' and entity.name != other_entity.name:
                    if len(self.humans)>0 or len(self.vampires)>0:
                        entity.interact_with_human(other_entity)
                    
                elif entity.type == 'vampire' and other_entity.type=='human':
                    if len(self.humans)>0 or len(self.vampires)>0:
                        entity.interact_with_human(other_entity)
                    
                elif entity.type == 'human' and other_entity.type=='vampire':
                    if len(self.humans)>0 or len(self.vampires)>0:
                        entity.interact_with_vampire(other_entity)
                    
                elif entity.type == 'vampire' and other_entity.type=='vampire' and entity.name != other_entity.name:
                    if len(self.vampires)>0:
                        entity.interact_with_vampire(other_entity)
                
                elif entity.type == 'human' and other_entity.type=='item':
                    if len(self.humans)>0:
                        entity.interact_with_item(other_entity)
                        
                elif entity.type == 'vampire' and other_entity.type=='item':
                    if len(self.vampires)>0:
                        entity.interact_with_item(other_entity)

    def is_adjacent(self, position1, position2):
        x1, y1 = position1
        x2, y2 = position2
        return abs(x1 - x2) <= 1 and abs(y1 - y2) <= 1

In [6]:
num_humans=50
num_vampires=20
time_steps=50
map_size=(100,100)
simulation = Simulation(num_humans, num_vampires, time_steps, map_size)
simulation.run_simulation()

--- Timestep 1 ---
Human bitten by the vampire: H21 turned into a vampire
Human bitten by the vampire: H41 turned into a vampire
Human bitten by the vampire: H21 turned into a vampire
Human bitten by the vampire: H41 turned into a vampire


48: Humans and 22: Vampires Left
--- Timestep 2 ---
Selfish interaction: H28 gained 20 health, H47 lost 20 health
Helpful interaction: H47 and H28 gained 10 health each
48: Humans and 22: Vampires Left
--- Timestep 3 ---
H12 gained 100 health from garlic
H18 gained 30 health from food
Human bitten by the vampire: H23 turned into a vampire
Human killed the vampire: H48 killed V17
Human bitten by the vampire: H23 turned into a vampire
Human bitten by the vampire: H48 turned into a vampire
46: Humans and 23: Vampires Left
--- Timestep 4 ---
Helpful interaction: H0 and H20 gained 10 health each
Helpful interaction: H5 and H42 gained 10 health each
Helpful interaction: H20 and H0 gained 10 health each
Human bitten by the vampire: H39 turned into a vampire
Helpful interaction: H42 and H5 gained 10 health each
Human bitten by the vampire: H39 turned into a vampire
45: Humans and 24: Vampires Left
--- Timestep 5 ---
45: Humans and 24: Vampires Left
--- Timestep 6 ---
H1 gained 30 health from f

## Replay Simulation

In [8]:
cap = cv2.VideoCapture('simulation.mp4')

def capture_video(cap):
    # Check if camera opened successfully 
    if (cap.isOpened()== False): 
        print("Error opening video file") 
      
    # Read until video is completed 
    while(cap.isOpened()): 
        # Capture frame-by-frame 
        ret, frame = cap.read() 
        if ret == True: 
        # Display the resulting frame 
            cv2.imshow('Frame', frame) 
            time.sleep(0.5)

              
        # Press Q on keyboard to exit 
            if cv2.waitKey(25) & 0xFF == ord('q'): 
                break
      
        # Break the loop 
        else: 
            break

      
    # When everything done, release 
    # the video capture object 
    cap.release() 
      
    # Closes all the frames 
    cv2.destroyAllWindows() 
capture_video(cap)